# Kaggle: San Francisco Crime Classification

Predict the category of crimes that occurred in the city by the bay

From 1934 to 1963, San Francisco was infamous for housing some of the world's most notorious criminals on the inescapable island of Alcatraz.

Today, the city is known more for its tech scene than its criminal past. But, with rising wealth inequality, housing shortages, and a proliferation of expensive digital toys riding BART to work, there is no scarcity of crime in the city by the bay.

From Sunset to SOMA, and Marina to Excelsior, this competition's dataset provides nearly 12 years of crime reports from across all of San Francisco's neighborhoods. Given time and location, you must predict the category of crime that occurred.

In [1]:
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cross_validation, preprocessing
from os.path import expanduser, normpath
import time
import datetime

# Gradient Tree Boosting
from sklearn.ensemble import GradientBoostingClassifier

# Random Forest
from sklearn.ensemble import RandomForestClassifier

from sklearn.grid_search import GridSearchCV

In [3]:
# Set paths for data to be imported

home = expanduser('~')
# path = str(home) + '\\Documents\\data-science\\kaggle\\sf-crime\\' # Windows
path = str(home) + '/Documents/Personal/Summagers/kaggle/sfcrime/mkchang/' # Mac
trainfile = 'train.csv'
testfile = 'test.csv'
train_gps_file = 'train_gps.csv'
test_gps_file = 'test_gps.csv'

In [4]:
train_data_raw = pd.read_csv(path+trainfile)
test_data_raw = pd.read_csv(path+testfile)
train_gps = pd.read_csv(path+train_gps_file)
test_gps = pd.read_csv(path+test_gps_file)

## Features

In [5]:
train_data = train_data_raw.copy()
test_data = test_data_raw.copy()

In [6]:
# Remove unnecessary features
train_data.drop(['Descript', 
                 'Resolution', 
                 'PdDistrict', 
                 'DayOfWeek', 
                 'Address'], inplace=True, axis=1)

test_data.drop(['PdDistrict', 
                'DayOfWeek', 
                'Address'], inplace=True, axis=1)

In [7]:
train_data['Dates'] = pd.to_datetime(train_data['Dates'])
train_data['year'] = train_data['Dates'].dt.year
train_data['month'] = train_data['Dates'].dt.month 
train_data['day'] = train_data['Dates'].dt.day
train_data['hour'] = train_data['Dates'].dt.hour
train_data['minute'] = train_data['Dates'].dt.minute

train_data['dayofyear'] = train_data['Dates'].dt.dayofyear
train_data['dayofweek'] = train_data['Dates'].dt.dayofweek

In [8]:
test_data['Dates'] = pd.to_datetime(test_data['Dates'])
test_data['year'] = test_data['Dates'].dt.year
test_data['month'] = test_data['Dates'].dt.month 
test_data['day'] = test_data['Dates'].dt.day
test_data['hour'] = test_data['Dates'].dt.hour
test_data['minute'] = test_data['Dates'].dt.minute

test_data['dayofyear'] = test_data['Dates'].dt.dayofyear
test_data['dayofweek'] = test_data['Dates'].dt.dayofweek

In [9]:
train_data['Z'] = train_gps['altitude (ft)']
test_data['Z'] = test_gps['altitude (ft)']

In [10]:
# remove training data with incorrect latitude and longitude
train_data = train_data[train_data['Y']!=90]

In [11]:
# Decide which features to go into training set
features = ['dayofyear','dayofweek','hour','X','Y','Z']

In [12]:
X_train = train_data.ix[:,features]
y_train = train_data.ix[:,'Category']
X_test = test_data.ix[:,features]

In [ ]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X_train, 
                                                                 y_train, 
                                                                 test_size=.3, 
                                                                 random_state=1)

In [ ]:
# # polarize data
#     if tod:
#         times = index.hour
#         tody = np.cos(2*np.pi*times/24)
#         todx = np.sin(2*np.pi*times/24)     
        
#         X_train[:,2] = tody[shuffling][:n_points]
#         X_train[:,3] = todx[shuffling][:n_points]
        
#         X_test[:,2] = tody[shuffling][n_points:]
#         X_test[:,3] = todx[shuffling][n_points:]

## Random Forest Model

In [13]:
crime_forest = RandomForestClassifier(n_estimators=50)

In [14]:
%time crime_forest = crime_forest.fit(X_train, y_train)

CPU times: user 2min 39s, sys: 10 s, total: 2min 49s
Wall time: 2min 56s


In [ ]:
%%time

score_train = crime_forest.score(X_train, y_train)
score_cv = crime_forest.score(X_cv, y_cv)

# test/train
# 20/80 split Training Score: 0.944199898638 , CV Score: 0.217073344343
# 50/50 split Training Score: 0.894782517584 , CV Score: 0.242728773988
# 80/20 split Training Score: 0.943824063687 , CV Score: 0.219235806617
print ('Training Score:', score_train, ', CV Score:', score_cv) 

In [15]:
# 50 trees, no max leaf nodes
crime_forest.feature_importances_

array([ 0.33612205,  0.11773859,  0.14074625,  0.13833773,  0.1442002 ,
        0.12285517])

In [16]:
prob_prediction = crime_forest.predict_proba(X_test)

In [17]:
submission = pd.DataFrame(prob_prediction, index=X_test.index, columns=crime_forest.classes_)

In [18]:
submission.to_csv('submission_2016_03_19.csv', index_label='Id')

### Plotting learning curves

In [ ]:
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
%%time 
X, y = X_train, y_train
train_data = X
# estimator = crime_forest

from sklearn.ensemble import GradientBoostingClassifier
estimator = GradientBoostingClassifier()
title = "Learning Curves (Random Forest)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = cross_validation.ShuffleSplit(train_data.shape[0], n_iter=5,
                                   test_size=0.3, random_state=0)

plot_learning_curve(estimator, title, X, y, n_jobs=4)
plt.show()

In [ ]:
# try gradient boosted
# change tree sizes
# add features (Z above sea level), replace hour with time of day
# try regularization (in ensemble) to correct overfitting
# voting classifier

### Exhaustive Grid Search

In [ ]:
# Split again, generate training and cross-validation features for grid search
X_grid_train, X_grid_cv, y_grid_train, y_grid_cv = cross_validation.train_test_split(X_train, 
                                                                                     y_train, 
                                                                                     test_size=0.40, 
                                                                                     random_state=1)

In [ ]:
param_grid = [
    {'n_estimators': [200], 'min_samples_split': [1, 2]}
]
scores = ['precision', 'recall']
# , 'max_features': [2, 3, 5]

In [ ]:
%%time
clf = GridSearchCV(RandomForestClassifier(), param_grid, error_score=0, n_jobs=1)
clf.fit(X_grid_train, y_grid_train)

print(clf.best_score_, clf.best_params_)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))

In [ ]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(OneVsRestClassifier(SVC()), param_grid,
                       scoring='%s_weighted' % score)
    clf.fit(X_grid_train, y_grid_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_grid_cv, clf.predict(X_grid_cv)
    print(classification_report(y_true, y_pred))
    print()